# LangChain
## ToolCalling
- LLM이 외부 기능이나 데이터에 접근할 수 있게 해주는 매커니즘
![](images/tool_call.png)

In [1]:
import re
import os, json
import warnings
from datetime import datetime
from pprint import pprint
from textwrap import dedent
from dotenv import load_dotenv
from langchain_core.messages import ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig

warnings.filterwarnings("ignore")
load_dotenv()

True

## 랭체인 내장 도구
- 랭체인은 검색, 코드 인터프리터, 생산성 도구 제공
### Tavily 도구 직접 사용해보기
- [docs](https://python.langchain.com/docs/integrations/tools/tavily_search/)

In [2]:
from langchain_community.tools import TavilySearchResults

query = "최근 한 달 동안 나온 테슬라와 구글 CEO 관련 영문 기사들을 각각 자연스럽게 한국어로 요약해서 Bullet Point로 정리해줘"

tavily_search = TavilySearchResults(max_results=2)
search_results = tavily_search.invoke(query)
search_results

[{'title': '구글 CEO의 고백, 테슬라가 최고다! 이유는? 인터뷰 번역, 댄 아이브스 ...',
  'url': 'https://www.youtube.com/watch?v=nu_9tQXlFBQ',
  'content': '구글 CEO의 고백, 테슬라가 최고다! 이유는? 인터뷰 번역, 댄 아이브스 예상주가 $515 간다! 테슬라 음성 서비스 시작하나  ? \n 오늘의 테슬라 뉴스 \n 3370 likes \n 57618 views \n 16 Dec 2024 \n ➤ 테슬라 구매하시나요? 추천링크 이동해서 33만원 할인을 받으세요.  ( 미국 $2천 달러 할인 )\n      추천링크 https://ts.la/sung85917\n\n➤ 오늘의 테슬라 뉴스 트위터 https://twitter.com/TodayTeslanews\n\n➤ 오늘의 테슬라 뉴스 회원가입https://www.youtube.com/c/%EC%98%A4%EB%8A%98%EC%9D%98%ED%85%8C%EC%8A%AC%EB%9D%BC%EB%89%B4%EC%8A%A4/join\n\n➤ Surfshark + 오테뉴 이벤트 4개월 추가 제공과 최저가 해택 놓치지 마세요! \n     https://surfshark.com/teslanews [...] ➤ 오늘의 테슬라 뉴스 광고 문의  todayteslanews@gmail.com\n\n➤욕설과 악플, 도배댓글, 타채널 홍보링크, 매너없는 글은 삭제/차단 합니다.\n➤주가 예측 질문 (살까요, 팔까요) 댓글에는 답변하지 않습니다.\n\n#테슬라 #테슬라주식 #테슬라뉴스 #ARK #일론머스크 #모델3  #모델y #investing #투자 #elonmusk #TSLA #Tesla #오테뉴 #트위터 #스페이스X #Twitter #오늘의테슬라뉴스 #주식투자 #해외주식 \n 235 comments',
  'score': 0.5852203},
 {'title': '[시사영어 1일1문] 머스크의 테슬라 사면초가에 (NYT ... - YouTube',
  '

### Tool의 구성 요소
- name : 도구의 이름
- description : 도구에 대한 설명
- JSON schema : 도구 입력 정보
- function : 실행할 함수??

In [3]:
TavilySearchResults # class 살펴보기
type(tavily_search)

langchain_community.tools.tavily_search.tool.TavilySearchResults

In [4]:
tavily_search.name

'tavily_search_results_json'

In [5]:
tavily_search.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [6]:
tavily_search.args

{'query': {'description': 'search query to look up',
  'title': 'Query',
  'type': 'string'}}

In [7]:
tavily_search.args_schema

langchain_community.tools.tavily_search.tool.TavilyInput

In [8]:
tavily_search.args_schema.model_json_schema()

{'description': 'Input for the Tavily tool.',
 'properties': {'query': {'description': 'search query to look up',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'TavilyInput',
 'type': 'object'}

### LLM과 Tavily 도구 binding하기

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=[tavily_search])

query

'최근 한 달 동안 나온 테슬라와 구글 CEO 관련 영문 기사들을 각각 자연스럽게 한국어로 요약해서 Bullet Point로 정리해줘'

In [10]:
ai_msg = llm_with_tools.invoke(query)
pprint(ai_msg)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QGyZBElYbnCnPWi9OFb54k0i', 'function': {'arguments': '{"query": "Tesla CEO news last month"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_aLSp5nL9jkFs6Ho6ViKYsXOT', 'function': {'arguments': '{"query": "Google CEO news last month"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 113, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BQYms5tdL4I1BkQVQ4GmnQxfq7BSs', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87ea773d-bca6-47f4-97d3-837e1bb849a7-0', tool_calls=[{'name': 'tavily_search_results_json', 'a

In [11]:
ai_msg.content

''

In [12]:
ai_msg.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'Tesla CEO news last month'},
  'id': 'call_QGyZBElYbnCnPWi9OFb54k0i',
  'type': 'tool_call'},
 {'name': 'tavily_search_results_json',
  'args': {'query': 'Google CEO news last month'},
  'id': 'call_aLSp5nL9jkFs6Ho6ViKYsXOT',
  'type': 'tool_call'}]

### LLM에서 응답으로 온 Tool 정보를 이용해 Tool 직접 실행하기
- args 스키마 사용 -> list 형태로 나옴
- tool_call 사용 -> ToolMessage 형태로 나옴

In [13]:
# args 스키마 사용
tool_call = ai_msg.tool_calls[0]
tool_output = tavily_search.invoke(tool_call['args'])
pprint(type(tool_output))
pprint(tool_output)

<class 'list'>
[{'content': "Tesla stock has spiked 20% since Musk said he's stepping back "
             'from DOGE\n'
             '\n'
             'With Tesla CEO Elon Musk set to return to Tesla after spending '
             'the last three months with the Trump administration, the stock '
             'is kicking into high gear.\n'
             '\n'
             'Shares of the electric vehicle maker have surged as much as 20% '
             'since Elon Musk announced plans to devote much more time to the '
             'company starting in May. [...] Some analysts are keeping their '
             'fingers crossed that this could be the start of an enduring '
             "recovery. Since Trump's inauguration, the Tesla CEO has made "
             'waves as the unofficial head of the Department of Government '
             "Efficiency, a role which has politicized Tesla's brand among "
             'consumers.',
  'score': 0.72278196,
  'title': "Tesla Stock Price Is up 20% Since 

In [14]:
# tool_call 사용
tool_message = tavily_search.invoke(tool_call)
pprint(type(tool_message))
pprint(tool_message)

<class 'langchain_core.messages.tool.ToolMessage'>
ToolMessage(content='[{"title": "Tesla Stock Price Is up 20% Since Musk Said He\'d Step Back From ...", "url": "https://www.businessinsider.com/tesla-stock-price-rally-elon-musk-doge-q1-earnings-tsla-2025-4", "content": "Tesla stock has spiked 20% since Musk said he\'s stepping back from DOGE\\n\\nWith Tesla CEO Elon Musk set to return to Tesla after spending the last three months with the Trump administration, the stock is kicking into high gear.\\n\\nShares of the electric vehicle maker have surged as much as 20% since Elon Musk announced plans to devote much more time to the company starting in May. [...] Some analysts are keeping their fingers crossed that this could be the start of an enduring recovery. Since Trump\'s inauguration, the Tesla CEO has made waves as the unofficial head of the Department of Government Efficiency, a role which has politicized Tesla\'s brand among consumers.", "score": 0.72278196}, {"title": "Tesla CEO 

In [15]:
# tool_output으로 ToolMessage 만들기
from langchain_core.messages.tool import ToolMessage

tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)
pprint(type(tool_message))
pprint(tool_message)

<class 'langchain_core.messages.tool.ToolMessage'>
ToolMessage(content=[{'title': "Tesla Stock Price Is up 20% Since Musk Said He'd Step Back From ...", 'url': 'https://www.businessinsider.com/tesla-stock-price-rally-elon-musk-doge-q1-earnings-tsla-2025-4', 'content': "Tesla stock has spiked 20% since Musk said he's stepping back from DOGE\n\nWith Tesla CEO Elon Musk set to return to Tesla after spending the last three months with the Trump administration, the stock is kicking into high gear.\n\nShares of the electric vehicle maker have surged as much as 20% since Elon Musk announced plans to devote much more time to the company starting in May. [...] Some analysts are keeping their fingers crossed that this could be the start of an enduring recovery. Since Trump's inauguration, the Tesla CEO has made waves as the unofficial head of the Department of Government Efficiency, a role which has politicized Tesla's brand among consumers.", 'score': 0.72278196}, {'title': 'Tesla CEO Musk says

In [16]:
tool_message.content

[{'title': "Tesla Stock Price Is up 20% Since Musk Said He'd Step Back From ...",
  'url': 'https://www.businessinsider.com/tesla-stock-price-rally-elon-musk-doge-q1-earnings-tsla-2025-4',
  'content': "Tesla stock has spiked 20% since Musk said he's stepping back from DOGE\n\nWith Tesla CEO Elon Musk set to return to Tesla after spending the last three months with the Trump administration, the stock is kicking into high gear.\n\nShares of the electric vehicle maker have surged as much as 20% since Elon Musk announced plans to devote much more time to the company starting in May. [...] Some analysts are keeping their fingers crossed that this could be the start of an enduring recovery. Since Trump's inauguration, the Tesla CEO has made waves as the unofficial head of the Department of Government Efficiency, a role which has politicized Tesla's brand among consumers.",
  'score': 0.72278196},
 {'title': 'Tesla CEO Musk says time he spends on DOGE will drop ... - YouTube',
  'url': 'http

### Tool의 결과를 LLM에 전달하여 최종 응답 생성하기

In [17]:
from datetime import datetime

today = datetime.today().strftime("%Y-%m-%d")
llm = ChatOpenAI(model="gpt-4o-mini")
prompt = f"""
    You are a helpful AI assistant.
    Today is {today}
    {query}
    {tool_message.content}
"""
result = llm.invoke(prompt)
pprint(result)

AIMessage(content='### 테슬라 관련 기사 요약\n- **테슬라 주가 20% 상승**: 일론 머스크가 DOGE에서 물러난 이후 테슬라 주가는 20% 상승했습니다. 머스크는 5월부터 회사에 더 많은 시간을 할애할 계획입니다.\n- **머스크, 정부 효율성 부서 활동 종료**: 머스크는 트럼프 행정부의 비공식 부서에서 활동한 후, 5월부터 테슬라에 집중하겠다고 밝혔습니다. 이로 인해 주가가 안정될 것이라는 기대가 커지고 있습니다.\n\n### 구글 CEO 관련 최근 기사 없음\n- 최근 한 달 동안 구글 CEO 관련 기사 목록에는 정보가 없습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 155, 'prompt_tokens': 416, 'total_tokens': 571, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BQYmzhcQX6nz5qhnPXbLHrrdpf4xo', 'finish_reason': 'stop', 'logprobs': None}, id='run-990b06cf-ba3a-48fc-8043-3f7ad7e013dc-0', usage_metadata={'input_tokens': 416, 'output_tokens': 155, 'total_tokens': 571, 'input_token_details': {'audio': 0

## 전체과정 정리 하기
- LLM 체인 정의
- LLM 체인에 ToolMessage 전달

In [18]:
# LLM 체인 정의

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant"),
    ("system", f"Today is {today}"),
    ("human", "{user_input}"),
    ("placeholder", "{messages}")
])

# llm 정의
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_search])

# LLM체인 생성
llm_chain = prompt | llm_with_tools


In [19]:
# LLM 체인에 ToolMessage 전달
from langchain_core.runnables import chain

@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input, config=config)
    tool_msgs = tavily_search.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke(
        {**input, "messages": [ai_msg, *tool_msgs]},
        config=config
    )

web_search_chain.invoke(query)

AIMessage(content='### 테슬라 CEO 관련 최근 기사 요약\n- **일론 머스크, DOGE에서 독립적 경영 계획 발표**: 머스크가 5월부터 테슬라에 더 많은 시간을 할애할 것이라고 발표함에 따라 회사 주가가 20% 상승. (출처: [Business Insider](https://www.businessinsider.com/tesla-stock-price-rally-elon-musk-doge-q1-earnings-tsla-2025-4))\n- **Musk, DOGE에 대한 시간 감소 예고**: 테슬라 CEO인 머스크가 4월 23일 기업 실적 발표에서 다음 달부터 DOGE와 관련된 사업에 투자하는 시간이 크게 줄어들 것이라고 밝혔음. (출처: [YouTube](https://www.youtube.com/watch?v=wPHVGYLZR8w))\n\n### 구글 CEO 관련 최근 기사 요약\n- **선더 피차이, 강력한 1분기 실적 보고**: 구글 및 알파벳 CEO인 피차이가 1분기 실적 발표에서 기업의 경쟁력과 혁신을 강조하며, 검색, 클라우드 및 유튜브에서의 건전한 성장세를 보고. 특히 AI 관련 신기능 도입과 구독 서비스에서 2억 7천만 건을 초과하는 유료 구독자를 기록했다고 언급. (출처: [Google Blog](https://blog.google/inside-google/message-ceo/alphabet-earnings-q1-2025/))\n- **피차이, 2025년의 높은 이해관계 강조**: 피차이가 직원들에게 회사가 경쟁 증가와 규제 장애물을 겪고 있다고 경고하며, 2025년의 중요성을 강조. (출처: [CNBC](https://www.cnbc.com/2024/12/27/google-ceo-pichai-tells-employees-the-stakes-are-high-for-2025.html))', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'com

## 사용자 정의 도구(Tool)
- @tool 데코레이터 사용

In [20]:
from typing import Dict, List
import requests
from langchain_core.tools import tool

@tool
def blog_search(query: str) -> List[Dict]:
    """네이버 블로그 API에 검색 요청을 보냅니다."""
    url = "https://openapi.naver.com/v1/search/blog.json"
    headers = {
        "X-Naver-Client-Id": os.environ["NAVER_CLIENT_ID"],
        "X-Naver-Client-Secret": os.environ["NAVER_CLIENT_SECRET"]
    }
    params = {"query": query, "display": 10, "start": 1}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()['items']
    else:
        return []

wine_query = "블로그에 나와있는 스테이크와 어울리는 와인과 기분 좋은 노래를 추천해주세요."
search_results = blog_search.invoke(wine_query)
search_results

[{'title': '이탈리아 피렌체 맛집 <b>추천</b> 티본<b>스테이크</b> 토스카나 와인 육회',
  'link': 'https://blog.naver.com/lsh5755/223847077153',
  'description': "피렌체 인근 토스카나 지방은 질 <b>좋은</b> <b>와인</b>을 생산하기로 유명하다. <b>와인</b> 애호가들의 성지기도 한다.... 그냥 'T본 <b>스테이크</b>'라고 부르고, 그들도 알아듣는다. T자 뼈가 포함되어 <b>있는</b> 안심과 등심 부위를... ",
  'bloggername': '천사는 여행중',
  'bloggerlink': 'blog.naver.com/lsh5755',
  'postdate': '20250426'},
 {'title': '<b>스테이크와</b> 잘 <b>어울리는</b> 3만원대 이마트 레드 <b>와인</b> <b>추천</b>',
  'link': 'https://blog.naver.com/dolmory9/223556006280',
  'description': '상태 <b>좋은</b> 코르크 : 산 페드로 시데랄 2021 ▲ <b>와인</b> 색상 : 산 페드로 시데랄 2021 (마트 토마스 글래스) &lt;음식 페어링&gt; 메인 육류 요리 또는 버섯과 잘 <b>어울리는</b> 레드 <b>와인</b>입니다. 등심 <b>스테이크와</b> 함께... ',
  'bloggername': '와인과 커피 엔지니어',
  'bloggerlink': 'blog.naver.com/dolmory9',
  'postdate': '20240821'},
 {'title': '여수 <b>와인</b>바 학동 맛집 :: <b>스테이크</b> 파스타 다이닝 바 와캡... ',
  'link': 'https://blog.naver.com/jhjoo20/223787083553',
  'description': '화이트 <b>와인</b>을 고를 때 직원분이 페어링하기 <b>좋은</b> 사이드로 <b>추천 

In [21]:
# LLM binding
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_blog = llm.bind_tools(tools=[tavily_search, blog_search])

In [22]:
# LLM 실행하여 툴 추천 받기
ai_msg = llm_with_blog.invoke(wine_query)
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xiSZUmpSYQCgiEnsaq0o4Scw', 'function': {'arguments': '{"query": "스테이크와 어울리는 와인"}', 'name': 'blog_search'}, 'type': 'function'}, {'id': 'call_PEHCb26U2pZU2wTstB5MLeyj', 'function': {'arguments': '{"query": "기분 좋은 노래 추천"}', 'name': 'blog_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 131, 'total_tokens': 189, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BQYnGPLpheMmq42QnyIvOHbEeQzjq', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c4f33ff6-187d-4bb2-bfe2-dac70581ed25-0', tool_calls=[{'name': 'blog_search', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_xiSZUmpSYQCgiEnsaq0o4Scw',

In [23]:
ai_msg.tool_calls

[{'name': 'blog_search',
  'args': {'query': '스테이크와 어울리는 와인'},
  'id': 'call_xiSZUmpSYQCgiEnsaq0o4Scw',
  'type': 'tool_call'},
 {'name': 'blog_search',
  'args': {'query': '기분 좋은 노래 추천'},
  'id': 'call_PEHCb26U2pZU2wTstB5MLeyj',
  'type': 'tool_call'}]

In [24]:
# blog tool 실행
blog_search.batch(ai_msg.tool_calls)

[ToolMessage(content='[{"title": "스미스앤월렌스키 (<b>스테이크와 어울리는</b> 추천<b>와인</b>, 서울... ", "link": "https://blog.naver.com/neotaboo/223758018557", "description": "해주셔서 <b>와인</b>과 함께 <b>스테이크</b> 먹기 너무 좋은 매장이었다. 식전빵이 나왔다. 로즈메리를 가득 뿌려... 잘 <b>어울리는</b> 수프였다. 꾸덕한 식감도 좋고, 풍부한 크림맛도 좋고. 입 짧은 나도 처음으로 접시 전체를... ", "bloggername": "입 짧은 콩나물", "bloggerlink": "blog.naver.com/neotaboo", "postdate": "20250212"}, {"title": "<b>스테이크와 어울리는 와인</b> 추천과 종류", "link": "https://blog.naver.com/oipicle31/223708302326", "description": "Information <b>스테이크</b> <b>어울리는 와인</b> 추천 <b>스테이크와</b> <b>와인</b>, 이 둘의 조합은 마치 해변의 파도와 모래처럼 완벽하게 <b>어울리는</b> 궁합이라고 할 수 있죠. 오늘은 여러분의 저녁 식사를 더욱 특별하게 만들어줄... ", "bloggername": "별난하루", "bloggerlink": "blog.naver.com/oipicle31", "postdate": "20241229"}, {"title": "경주 황리단길 파스타 벚꽃과 정말 잘 <b>어울리는</b> <b>스테이크</b> <b>와인</b>... ", "link": "https://blog.naver.com/ljy213/223809691723", "description": "경주 황리단길 파스타 맛집 또봄은 이탈리아 음식점으로써 파스타와 <b>스테이크</b> 리조또 등의 메뉴가 주가 있었고, 다양한 <b>와인</b>이 있어서 그런지 메뉴판의 상당부분이 <b